# I. Import the Dataset

### Step 1. Import the necessary libraries

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np


### Step 2. Load the digits.csv dataset 
This is a copy of the test set of the UCI ML hand-written digits datasets https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where each class refers to a digit.

Each datapoint is a 8x8 image of a digit, Classes=10, Samples per class~180, 

Samples total=1797, Dimensionality=64, and Features=(integers 0-16)


- After importing this dataset, split it into test and train sets. 
- You may check the shape of data and the target attributes of the dataset. 
- You may also want to print a few samples from the dataset.

In [10]:
data = pd.read_csv('digits.csv', header=None)

# Separate features and target
X = data.iloc[:, :-1]  # Features (64 columns)
y = data.iloc[:, -1]   # Target (last column)

# X, y

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the data and target
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)




Shape of X_train: (1438, 64)
Shape of y_train: (1438,)
Shape of X_test: (360, 64)
Shape of y_test: (360,)


# II. Cross-validation

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called overfitting. To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set X_test, y_test. 

When evaluating different settings (“hyperparameters”) for estimators, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

A model is trained using  of the folds as training data; the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data (as is the case when fixing an arbitrary validation set), which is a major advantage in problems such as inverse inference where the number of samples is very small.
(Check https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)

### Step 1. Import libraries
Import the classes `cross_val_score` and `KNeighborsClassifier` 

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


### Step 2. Run cross validation
Run cross validation for `cv=5` on the train dataset. Print the scores.

In [ ]:
# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform k-fold cross-validation
k = 5
cv_scores = cross_val_score(model, X_train, y_train, cv=k)


### Step 3: Print the mean score and confidence interval
Print the mean score of cross validation with 95% confidence interval.

In [ ]:
# Calculate mean and standard error
mean_score = np.mean(cv_scores)
std_error = np.std(cv_scores) / np.sqrt(cv)

# Calculate 95% confidence interval
confidence_interval = stats.norm.interval(0.95, loc=mean_score, scale=std_error)

# Print the results
print("Mean cross-validation score:", mean_score)
print("95% Confidence interval:", confidence_interval)


### Step 4. Repeat Step 3 for  KFold, StratifiedKFold
Repeat Step 3 for `KFold`, `StratifiedKFold` with number of splits `n_splits=10`. Print the mean score with 95% confidence interval.

In [ ]:
# Function to perform cross-validation and calculate mean score with confidence interval
def evaluate_cross_validation(cv_strategy, X_train, y_train):
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv_strategy)
    mean_score = np.mean(cv_scores)
    std_error = np.std(cv_scores) / np.sqrt(len(cv_scores))
    confidence_interval = stats.norm.interval(0.95, loc=mean_score, scale=std_error)
    return mean_score, confidence_interval

# KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
mean_kfold, ci_kfold = evaluate_cross_validation(kfold, X_train, y_train)
print("KFold Mean cross-validation score:", mean_kfold)
print("KFold 95% Confidence interval:", ci_kfold)

# StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
mean_stratified_kfold, ci_stratified_kfold = evaluate_cross_validation(stratified_kfold, X_train, y_train)
print("StratifiedKFold Mean cross-validation score:", mean_stratified_kfold)
print("StratifiedKFold 95% Confidence interval:", ci_stratified_kfold)


Grid Searches
=================
Exhaustive search over specified parameter values for an estimator.

Important members are fit, predict.

GridSearchCV implements a “fit” and a “score” method. It also implements “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

See (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

### Step 1. Grid-Search with build-in cross validation
Import `GridSearchCV` from `sklearn.model_selection` and `SVC` from `sklearn.svm`

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

### Step 2: Define parameter grid:
- `SVC` has two parameters `C` and `gamma`. 
- Do not worry about the roles of these two parameters on the algorithm. 
- Set model parameters to `'C': array([  0.001, 0.01,0.1,1,10,100])` and `gamma': array([0.00001,0.0001,0.001, 0.01, 0.1])`.
- There are different ways of assigning parameter values, your instructor will lead you.

In [17]:
import numpy as np

param_grid = {
    'C': np.array([0.001, 0.01, 0.1, 1, 10, 100]),
    'gamma': np.array([0.00001, 0.0001, 0.001, 0.01, 0.1])
}

### Step 3. Instintiate the grid search object
Instintiate a grid search object for `SVC`, for the corresponding paranmeters with 5-folds cross validation.

In [18]:
svc = SVC()

grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5)


### Step 4. Fit the grid search object on the train data set
A GridSearchCV object behaves just like a normal classifier.

In [ ]:
grid_search.fit(X_train, y_train)

### Step 5. Grid Search Results
- Form a data frame from the grid search results.
- Print the first 10 predictions of `X_test` using the grid search model.
- Print the best parameter, best score and estimator of the grid search.
- Create a heat map for the grid search parameters using the parameter values and the grid search results.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Form a DataFrame from the grid search results
results_df = pd.DataFrame(grid_search.cv_results_)

# 2. Print the first 10 predictions of X_test using the grid search model
predictions = grid_search.predict(X_test)
print("First 10 Predictions:", predictions[:10])

# 3. Print the best parameters, best score, and the best estimator
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
print("Best Estimator:", grid_search.best_estimator_)

# 4. Create a heat map for the grid search parameters
# Pivot the DataFrame for the heatmap
heatmap_data = results_df.pivot("param_gamma", "param_C", "mean_test_score")

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt=".3f", cmap="viridis")
plt.title("Grid Search Mean Test Scores")
plt.xlabel("C Parameter")
plt.ylabel("Gamma Parameter")
plt.show()